In [16]:
sc

In [17]:
dfTweets = sqlContext.read.load('samplestream_twt.jsonl', format='json')
dfTweets.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- media_url: string (nulla

In [18]:
# status = dfTweets.select('retweeted_status.entities.urls.url', 'retweeted_status.id_str','retweeted_status.lang','retweeted_status.favorite_count', 'retweeted_status.retweet_count','retweeted_status.text')
status = dfTweets.select('retweeted_status.entities.urls.url', 'retweeted_status.id_str','retweeted_status.lang','retweeted_status.favorite_count', 'retweeted_status.retweet_count','retweeted_status.text')

status.show()

+--------------------+------------------+----+--------------+-------------+--------------------+
|                 url|            id_str|lang|favorite_count|retweet_count|                text|
+--------------------+------------------+----+--------------+-------------+--------------------+
|                  []|862314907819180032|  en|         16006|         6987|President Trump -...|
|[https://t.co/ldj...|862434817727766528|  es|             0|            3|#AHORA | Director...|
|                  []|862363801709985792|  en|           735|          272|WH STAFF: Dude th...|
|[https://t.co/cWw...|862409155222622210|  en|          1403|          742|Why it's not craz...|
|                null|              null|null|          null|         null|                null|
|[https://t.co/cQw...|862394567102001152|  en|           593|          440|NBC's @SRuhle: An...|
|                  []|862112264266043392|  en|        111865|        40119|I miss the days w...|
|                  []|86238469

In [19]:
retweet_status = dfTweets.select('place.bounding_box', 'place.full_name')
retweet_status.registerTempTable('toplocations')
occurrence = sqlContext.sql('select bounding_box, full_name, count(full_name) as occurrence from toplocations group by full_name, bounding_box')\
.dropna() #.limit(50)  #limitby the most 50 occurrence
toploc = occurrence.orderBy('occurrence', ascending=False) #order by descending order based on occurrence

In [52]:
# toploc.write.csv('toplocations/toplocations.csv')
#toploc.write.format('csv').save('toplocations/toplocations.csv')


# This commands will not work on cluster. Doesnt return a csv file.
#toploc.toPandas().to_csv('toplocations/toplocations.csv', encoding='utf-8')

In [36]:
# This command is what we use in the cluster.
toploc.rdd.saveAsTextFile('finaltoplocationsout.csv')

## After running the code above on cluster

In [186]:
toplocations = sc.textFile("finaltoplocationsout.csv")

In [187]:
toplocations.take(1)

[u"Row(bounding_box=Row(coordinates=[[[-95.823268, 29.522325], [-95.823268, 30.154665], [-95.069705, 30.154665], [-95.069705, 29.522325]]], type=u'Polygon'), full_name=u'Houston, TX', occurrence=5)"]

In [188]:
remove = toplocations.map(lambda x: x.replace("Row(bounding_box=Row(coordinates=[[[", ""))
remove = remove.map(lambda x: x.replace("]], type=u'Polygon'), full_name=u'", ","))
remove = remove.map(lambda x: x.replace(", occurrence=", ";"))
final = remove.map(lambda x: x.replace(")", ", NULL").replace("],", ';').replace('[', '').replace(']', '').replace("'", ''))

final.take(2)

[u'-95.823268, 29.522325; -95.823268, 30.154665; -95.069705, 30.154665; -95.069705, 29.522325;Houston, TX;5, NULL',
 u'-74.026675, 40.683935; -74.026675, 40.877483; -73.910408, 40.877483; -73.910408, 40.683935;Manhattan, NY;5, NULL']

In [189]:
def mapper(k1v1s): 
    for k1v1 in k1v1s:
        k = k1v1.split(';')
        x_co = float(k[0].split(',')[0]) + float(k[1].split(',')[0])+ float(k[2].split(',')[0])+float(k[3].split(',')[0])
        x_co = x_co/4
        y_co = float(k[0].split(',')[1]) + float(k[1].split(',')[1])+ float(k[2].split(',')[1])+float(k[3].split(',')[1])
        y_co = y_co/4
        yield (round(y_co,6), round(x_co, 6))

coordinates = final.mapPartitions(mapper)
coordinates.take(20)

[(29.838495, -95.446486),
 (40.780709, -73.968542),
 (39.154153, -76.704632),
 (31.168893, -100.076888),
 (32.678207, -83.173867),
 (43.629311, -79.27257),
 (33.47601, -86.927537),
 (30.323346, -97.754724),
 (32.810012, -117.104989),
 (34.020789, -118.411907),
 (34.751936, -92.131274),
 (38.003376, -79.420865),
 (42.746617, -75.770041),
 (27.698682, -83.804475),
 (33.552863, -112.124676),
 (43.056722, -87.967292),
 (32.189391, -110.915573),
 (48.088757, -122.164528),
 (37.497293, -122.232934),
 (38.920142, -80.181835)]

In [ ]:
# def mapper(k1v1s): 
#     for k1v1 in k1v1s:
#         k = k1v1.split(';')[0]
#         co = k.split(', ')
#         try:
#             yield (float(co[0]), float(co[1]))
#         except:
#             pass
# #                yield (co[0], co[1]) # if uncomment then shows location and count

# coordinates = final.mapPartitions(mapper)
# coordinates.take(20)


# [(-95.823268, 29.522325),
#  (-95.823268, 30.154665),
#  (-95.069705, 30.154665),
#  (-95.069705, 29.522325),
#  (-74.026675, 40.683935),
#  (-74.026675, 40.877483),

In [192]:
import csv

def writeRecords(records):
    with open('coordinates.csv', 'w') as csvfile:
        fieldnames = ['lat', 'long']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for record in records:
            writer.writerow({'lat': record[0], 'long': record[1]})

In [193]:
coordinates = coordinates.toLocalIterator()
writeRecords(coordinates)

AttributeError: 'generator' object has no attribute 'toLocalIterator'